In [1]:
import sys
sys.path = ['/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/RecStudio'] + sys.path
import os
import logging

import faiss
import numpy as np
import pandas as pd 
from tqdm import tqdm
import pickle
from recstudio.data.advance_dataset import KDDCUPSessionDataset

/root/autodl-tmp/xiaolong/anaconda3/envs/pytorch_gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
logger = logging.getLogger(__name__)

In [9]:
def _load_cache(path):
    with open(path, 'rb') as f:
        download_obj = pickle.load(f)
    return download_obj

def load_datasets_from_cache(data_dir):
    logger.info(f"Load dataset from cache {data_dir}.")
    cache_datasets = _load_cache(data_dir)
    datasets = []
    for i in range(len(cache_datasets)):
        datasets.append(KDDCUPSessionDataset(None, data_dir, None, True))
        for k in cache_datasets[i].__dict__:
            attr = getattr(cache_datasets[i], k)
            setattr(datasets[i], k, attr)
    return datasets 

In [10]:
dataset = load_datasets_from_cache(
        '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/.recstudio/cache/f30c8c1234f63ba5c60a8b7814777f60'
)[0]

In [13]:
product_data = pd.read_csv('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv')

In [14]:
product_data

,id,locale,title,price,brand,color,size,model,material,author,desc
0,B005ZSSN10,DE,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,30.95,RED DRAGON,NaN,NaN,RDD0089,NaN,NaN,Amberjacks Steel Dartpfeile sind verfügbar in ...
1,B08PRYN6LD,DE,Simply Keto Lower Carb* Schokodrops ohne Zucke...,17.90,Simply Keto,NaN,750 g (1er Pack),NaN,NaN,NaN,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
2,B09MBZJ48V,DE,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",68.89,Sennheiser,Multi-Colour,One size,508377,Kunstleder,NaN,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
3,B08ZN6F26S,DE,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,18.99,Amy & Benton,Animal Car,NaN,2008B,aufziehauto 1 jahr,NaN,【Auto aufziehbar】: Drücken Sie einfach leicht ...
4,B094DGRV7D,DE,PLAYMOBIL - 70522 - Cavaliere mit grauem Pony,7.17,PLAYMOBIL,Nicht Zutreffend.,OneSize,70522,Polypropylen,NaN,Inhalt: 1 Stück
...,...,...,...,...,...,...,...,...,...,...,...
1551052,B09BW5CDRR,IT,Barbie - Playset Gelateria con Bambola con Mac...,20.48,Barbie,NaN,NaN,HCN46,NaN,NaN,DETTAGLI REALISTICI. Basta inserire la pasta m...
1551053,B0050IILBM,IT,"Braun Silk-épil 1 Depilatore Donna, Epilatore ...",22.61,Braun,Pink,NaN,4210201656067,NaN,NaN,Alimentato a corrente per un comodo utilizzo
1551054,B07W4C5W9D,IT,BoxLegend Sacchetti Sottovuoto Vestiti 6 Pezzi...,14.99,BoxLegend,6 Pezzi.,6 Pezzi (2L + 2M + 2S),6186666487608_SML,Polietilene Ppa,NaN,6 Sacchetti in 3 Diverse Misure- Questo set di...
1551055,B012D0HJXA,IT,Trasportino Pratiko Metal - Accessorio da viag...,18.35,MPS,verde,NaN,NaN,Metallo,NaN,TRASPORTINO 48X31.5X33CM


In [12]:
dataset.item_all_data

,product_id,locale,title,price,brand,color,size,model,material,author,desc
0,0,0,,1.806107e+06,,,,0,,,
1,1168061,2,RED DRAGON Amberjack 3 - Steel Tip 22 Gramm Wo...,3.095000e+01,RED DRAGON,,,1,,,Amberjacks Steel Dartpfeile sind verfügbar in ...
2,920036,2,Simply Keto Lower Carb* Schokodrops ohne Zucke...,1.790000e+01,Simply Keto,,750 g (1er Pack),0,,,🌱 NATÜRLICHE SÜSSE DURCH ERYTHRIT - Wir stelle...
3,267240,2,"Sennheiser 508377 PC 5.2 Chat, Stilvolles Mult...",6.889000e+01,Sennheiser,Multi-Colour,One size,2,Kunstleder,,3.5 MM BUCHSE - Kann problemlos an Geräte mit ...
4,657757,2,AmyBenton Auto ab 1 2 3 ahre - Baby Aufziehbar...,1.899000e+01,Amy & Benton,Animal Car,,3,aufziehauto 1 jahr,,【Auto aufziehbar】: Drücken Sie einfach leicht ...
...,...,...,...,...,...,...,...,...,...,...,...
1551053,135706,6,Barbie - Playset Gelateria con Bambola con Mac...,2.048000e+01,Barbie,,,490797,,,DETTAGLI REALISTICI. Basta inserire la pasta m...
1551054,132151,6,"Braun Silk-épil 1 Depilatore Donna, Epilatore ...",2.261000e+01,Braun,Pink,,29156,,,Alimentato a corrente per un comodo utilizzo
1551055,31791,6,BoxLegend Sacchetti Sottovuoto Vestiti 6 Pezzi...,1.499000e+01,BoxLegend,6 Pezzi.,6 Pezzi (2L + 2M + 2S),114683,Polietilene Ppa,,6 Sacchetti in 3 Diverse Misure- Questo set di...
1551056,1381524,6,Trasportino Pratiko Metal - Accessorio da viag...,1.835000e+01,MPS,verde,,0,Metallo,,TRASPORTINO 48X31.5X33CM


In [3]:
item_reps = np.load('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/text_method/roberta_results/results_epoch_4/item_reps/item.npy')

In [5]:
item_reps.shape

(1551057, 768)

In [7]:
item_reps.dtype

dtype('float32')